In [15]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import Imputer
import numpy as np

from PreprocessingKeyWords2 import preprocessing
preprocessor=preprocessing()

from Word2Vec import Word2Vec2
word2vecmodel=Word2Vec2()
word_model=word2vecmodel.loadWord2Vec()

In [16]:
train_data=pd.read_csv("trainData.csv")

#Data Preprocessing

#removing the hmid
train_data = train_data.drop(['hmid','concepts'], axis=1)

#remove missing values
train_data=train_data.replace(('prefer not to say'),np.NAN)
#print(train_data.isnull().sum())
train_data=train_data.dropna()

#print(train_data.isnull().sum())
# print(len(train_data))

#creating features for moments data
# happycorpus=preprocessor.getSocialKeywords(train_data['moment'])
happycorpus=preprocessor.getAgencyKeywords(train_data['moment'])

featureMatrix=word2vecmodel.getFeatureMatrix(word_model,happycorpus)

In [23]:
y_agency = train_data.agency
y_social=train_data.social
print("Step 1:",type(y_social))

# #train n test data split
# train_X, test_X, train_y, test_y = train_test_split(featureMatrix, y_social, test_size=0.25)
# print("done !!")

train_X, test_X, train_y, test_y = train_test_split(featureMatrix, y_agency, test_size=0.25)
print("done !!")


Step 1: <class 'pandas.core.series.Series'>
done !!


In [24]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
model = XGBClassifier()
model.fit(train_X, train_y)
print(model)
# make predictions for test data
y_pred = model.predict(test_X)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)


In [25]:
print(y_pred)
predictions = [value for value in y_pred]
# evaluate predictions
accuracy = accuracy_score(test_y, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

['yes' 'yes' 'yes' ... 'yes' 'yes' 'yes']
Accuracy: 82.22%
